# Prepare data

## Import Libraries

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA

from xgboost import XGBRegressor, plot_importance

from datetime import date, timedelta, datetime
import itertools


%matplotlib inline

## Load data

In [4]:
data_path = '../data'

# wf  = pd.read_csv(f'{data_path}/Historical_Wildfires.csv')
wf = pd.read_csv(f'{data_path}/Historical_Wildfire_fill0.csv')
wt_  = pd.read_csv(f'{data_path}/HistoricalWeather.csv')
df3 = pd.read_csv(f'{data_path}/HistoricalWeatherForecasts.csv')
df4 = pd.read_csv(f'{data_path}/LandClass.csv')
df5 = pd.read_csv(f'{data_path}/VegetationIndex.csv')

In [5]:
wf

,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Region
0,2005-01-01,0.000000,0.000000,0.000000,0.000000,NSW
1,2005-01-02,0.000000,0.000000,0.000000,0.000000,NSW
2,2005-01-03,0.000000,0.000000,0.000000,0.000000,NSW
3,2005-01-04,8.680000,312.266667,42.400000,78.666667,NSW
4,2005-01-05,16.611250,322.475000,62.362500,85.500000,NSW
...,...,...,...,...,...,...
40476,2020-10-27,32.970000,324.068182,103.318182,84.363636,WA
40477,2020-10-28,20.840625,314.425000,37.256250,88.312500,WA
40478,2020-10-29,136.083077,323.588461,294.102564,92.974359,WA
40479,2020-10-30,42.397895,324.205263,54.994737,89.631579,WA


In [6]:
wt_

,Date,Region,Parameter,count()[unit: km^2],min(),max(),mean(),variance()
0,2005-01-01,NSW,Precipitation,8.002343e+05,0.000000,1.836935,0.044274,0.028362
1,2005-01-01,NSW,RelativeHumidity,8.002343e+05,13.877194,80.522964,36.355567,253.559937
2,2005-01-01,NSW,SoilWaterContent,8.002343e+05,0.002245,0.414305,0.170931,0.007758
3,2005-01-01,NSW,SolarRadiation,8.002343e+05,14.515009,32.169781,26.749389,6.078587
4,2005-01-01,NSW,Temperature,8.002343e+05,14.485785,35.878704,27.341182,18.562212
...,...,...,...,...,...,...,...,...
242776,2020-10-31,WA,RelativeHumidity,2.528546e+06,9.272744,82.022224,34.193973,309.018764
242777,2020-10-31,WA,SoilWaterContent,2.528546e+06,0.000000,0.285399,0.050737,0.001419
242778,2020-10-31,WA,SolarRadiation,2.528546e+06,17.072290,29.743744,26.746413,5.986063
242779,2020-10-31,WA,Temperature,2.528546e+06,16.238239,34.669132,28.090392,15.293953


## Format weather data to Day/refion base

In [7]:
def trans_Hist_Weather(df):

    df = df.rename(columns={'count()[unit: km^2]': 'Area', 'min()': 'Min',
                            'max()': 'Max', 'mean()': 'Mean', 'variance()': 'Variance'})
    df = df.pivot_table(values=['Min', 'Max', 'Mean', 'Variance'], index=['Date', 'Region'],
                        columns=['Parameter'])
    df.reset_index(inplace=True)
    df.columns = [c[0] if c[1] ==
                  '' else f'{c[1]}_{c[0]}' for c in df.columns.values]
    df['Date'] = pd.to_datetime(df['Date'])

    return df

In [8]:
wt = trans_Hist_Weather(wt_)

In [9]:
wt

,Date,Region,Precipitation_Max,RelativeHumidity_Max,SoilWaterContent_Max,SolarRadiation_Max,Temperature_Max,WindSpeed_Max,Precipitation_Mean,RelativeHumidity_Mean,...,SoilWaterContent_Min,SolarRadiation_Min,Temperature_Min,WindSpeed_Min,Precipitation_Variance,RelativeHumidity_Variance,SoilWaterContent_Variance,SolarRadiation_Variance,Temperature_Variance,WindSpeed_Variance
0,2005-01-01,NSW,1.836935,80.522964,0.414305,32.169781,35.878704,7.670482,0.044274,36.355567,...,0.002245,14.515009,14.485785,1.354448,0.028362,253.559937,0.007758,6.078587,18.562212,0.850048
1,2005-01-01,NT,315.266815,95.683342,0.496140,31.634459,38.136787,9.704402,9.884958,61.494675,...,0.000000,2.518120,24.179960,1.840394,546.059262,584.201131,0.026743,58.942658,12.920252,1.930014
2,2005-01-01,QL,74.452164,95.898270,0.472416,31.982830,37.047943,7.675632,1.453053,47.959364,...,0.000000,6.033827,20.951620,1.106028,35.641257,403.134377,0.012679,29.500832,13.792599,0.883048
3,2005-01-01,SA,3.193624,81.980751,0.263911,31.734528,38.326847,10.044715,0.059078,30.057683,...,0.000000,17.861103,14.095855,2.023657,0.042837,246.044713,0.001917,7.914246,34.799336,1.655908
4,2005-01-01,TA,13.604791,81.501442,0.368189,33.225517,16.228510,11.432408,3.099497,65.086764,...,0.000000,20.742302,6.686816,1.995647,12.068597,111.754034,0.007121,12.826400,4.912013,2.963118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40476,2020-10-31,QL,26.366877,87.803581,0.442840,30.670065,32.112972,7.643718,0.386800,35.829668,...,0.000000,19.898029,17.249844,1.180556,4.229566,316.368911,0.006935,4.796284,8.811943,0.549077
40477,2020-10-31,SA,0.259189,82.642616,0.436347,29.623133,26.117023,7.620720,0.006940,45.700218,...,0.000000,15.608338,9.948414,2.062619,0.000492,316.308826,0.005425,17.093209,9.466862,0.640926
40478,2020-10-31,TA,2.179307,86.428932,0.376833,28.439209,14.577375,7.388186,0.472709,77.645878,...,0.000000,15.503663,8.217092,1.300771,0.243603,10.936866,0.009317,8.718983,2.388990,0.859296
40479,2020-10-31,VI,11.436618,93.374763,0.455111,28.041906,17.307510,6.605598,0.822028,78.326366,...,0.000000,11.170260,9.186510,1.783996,3.148454,44.766480,0.005050,9.917196,4.088503,1.019079


In [10]:
wf['Date'] = pd.to_datetime(wf['Date'])

In [20]:
wf

,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Region
0,2005-01-01,0.000000,0.000000,0.000000,0.000000,NSW
1,2005-01-02,0.000000,0.000000,0.000000,0.000000,NSW
2,2005-01-03,0.000000,0.000000,0.000000,0.000000,NSW
3,2005-01-04,8.680000,312.266667,42.400000,78.666667,NSW
4,2005-01-05,16.611250,322.475000,62.362500,85.500000,NSW
...,...,...,...,...,...,...
40476,2020-10-27,32.970000,324.068182,103.318182,84.363636,WA
40477,2020-10-28,20.840625,314.425000,37.256250,88.312500,WA
40478,2020-10-29,136.083077,323.588461,294.102564,92.974359,WA
40479,2020-10-30,42.397895,324.205263,54.994737,89.631579,WA


## Finding missing date

In [12]:
def find_missing_date(df, start_date = (2005,1,1), end_date = (2020,10,31), 
                      region = ['NSW', 'NT', 'QL', 'SA', 'TA', 'VI', 'WA'], date_col = 'Date') :
    
    df['Date'] = pd.to_datetime(df['Date'])
    start_date = date(start_date[0], start_date[1], start_date[2])
    end_date = date(end_date[0], end_date[1], end_date[2])
    total_date_set = set([start_date + timedelta(x) for x in range((end_date- start_date).days)])
    
    missing = dict()
    for reg in region : 
        df_temp = df[df['Region'] == reg]
        date_list = list(df_temp[date_col])
        date_set = set([ts.date() for ts in date_list])
        
        missing[reg] = total_date_set - date_set
        
    return missing

In [13]:
find_missing_date(wf, region = ['NSW'])

{'NSW': set()}

In [14]:
find_missing_date(wt, region = ['NSW'])

{'NSW': set()}

In [15]:
def gen_supervise(df, ft_cols=None, tgt_col='Estimated_fire_area', lag=7, forecast=30):

    ft = list(df.values)
    tgt = list(df[tgt_col].values)

    ft = [ft[i:i+lag] for i in range(len(tgt)-forecast-lag+1)]
    ft = np.array([list(itertools.chain(*temp)) for temp in ft])
    target = np.array([tgt[i: i + forecast] for i in range(lag, len(tgt)-forecast+1)])

    return ft, target


def prepared_data(df, split=0.8, lag=30, forecast=30, ft_cols=None):

    if ft_cols == None:
        ft_cols = [ft for ft in df.columns.values if ft not in [
            'Date', 'Region', 'Replaced']]
    ttl_rows = len(df.index.values)
    temp_df = df[ft_cols]

    split_idx = round(split*ttl_rows)
    scaler = MinMaxScaler(feature_range=(0, 1))
    train, test = temp_df.iloc[:split_idx, :], temp_df.iloc[split_idx:, :]

    scaled_train = scaler.fit_transform(train)
    scaled_train_df = pd.DataFrame(scaled_train, columns=ft_cols)
    scaled_test = scaler.transform(test)
    scaled_test_df = pd.DataFrame(scaled_test, columns=ft_cols)

    scaled_data = dict()
    scaled_data['X_train'], scaled_data['y_train'] = gen_supervise(
        scaled_train_df, lag=lag, forecast=forecast, ft_cols=ft_cols)
    scaled_data['X_test'], scaled_data['y_test'] = gen_supervise(
        scaled_test_df, lag=lag, forecast=forecast)

    data = dict()
    data['X_train'], data['y_train'] = gen_supervise(
        train, lag=lag, forecast=forecast, ft_cols=ft_cols)
    data['X_test'], data['y_test'] = gen_supervise(
        test, lag=lag, forecast=forecast, ft_cols=ft_cols)

    return scaler, data, scaled_data, train, test, scaled_train_df, scaled_test_df

In [26]:
def xgb_model(df, region, ft_cols=None, fore=29):

    if ft_cols == None:
        ft_cols = [ft for ft in df.columns.values if ft not in [
            'Date', 'Region', 'Replaced']]

    scaler, data, scaled_data, train, _, _, _ = prepared_data(
        df[df['Region'] == region], forecast=30, lag=60, ft_cols=ft_cols)
    
    print(train)
    
    X_train = pd.DataFrame(scaled_data['X_train'])
    X_test = pd.DataFrame(scaled_data['X_test'])
    y_train = pd.DataFrame(scaled_data['y_train'])
    y_test = pd.DataFrame(scaled_data['y_test'])

    xgr = XGBRegressor()
    xgr.fit(X_train, y_train[fore])
    preds = xgr.predict(X_test)
    preds_tr = xgr.predict(X_train)
    RMSE = np.sqrt(mean_squared_error(y_test[fore], preds))
    RMSE_tr = np.sqrt(mean_squared_error(y_train[fore], preds_tr))
    MAE = mean_absolute_percentage_error(y_test[fore], preds)
    MAE_tr = mean_absolute_percentage_error(y_train[fore], preds_tr)
    
    print(f'Region : {region}')
    print(f'RMSE(test)  : {RMSE}')
    print(f'RMSE(train) : {RMSE_tr}')
    print(f'MAE(test)   : {MAE}')
    print(f'MAE(train)  : {MAE_tr}')
    print('----------------------------------------------')
    
    y = {'y_train':y_train, 'y_test':y_train}

    return xgr, RMSE, scaler, preds, preds_tr, data, y_test, y_train

In [27]:
def train_visualize(df):
    pred_dict = dict()
#     pred_list = list()
    for r in df.Region.unique().tolist():
        model, RMSE, scaler, preds, preds_tr, data, y_test, y_train = xgb_model(
            df, region=r)
        y_join = pd.concat([y_train[[6]], y_test[[6]]]).reset_index(drop=True)
        y_join = y_join.rename(columns={6: 'tgt'})
        y_join = df[df['Region'] == r][[
            'Date']].reset_index(drop=True).join(y_join, how='inner')

        preds = pd.DataFrame(preds).rename(columns={0: 'pred'})
        preds_tr = pd.DataFrame(preds_tr).rename(columns={0: 'pred'})
        preds_join = pd.concat([preds_tr, preds])
        preds_join = preds_join.reset_index(drop=True).rename(columns={0: 'pred'})

        preds = preds_join[preds_tr.shape[0]:].rename(columns={'pred': 'test'})

        y_join = y_join.join(preds_tr)
        y_join = y_join.join(preds)
        y_join = y_join.rename(columns={'pred': 'train'})

        fig, ax = plt.subplots(figsize=(20, 5))
        sns.lineplot(x='Date', y='tgt', data=y_join,
                     color='r', alpha=0.5, label="Real")
        sns.lineplot(x='Date', y='train', data=y_join,
                     color='b', alpha=0.5, label="train")
        sns.lineplot(x='Date', y='test', data=y_join,
                     color='g', alpha=0.5, label="test")
        pred_dict[r] = y_join
#         pred_list.append(y_join)
        
    return pred_dict, y_join

In [ ]:
pred_dict, y_join = train_visualize(wf)

      Estimated_fire_area  Mean_estimated_fire_brightness  \
0                0.000000                        0.000000   
1                0.000000                        0.000000   
2                0.000000                        0.000000   
3                8.680000                      312.266667   
4               16.611250                      322.475000   
...                   ...                             ...   
4621            20.876471                      315.167647   
4622            29.633333                      310.550000   
4623            72.338824                      310.788235   
4624            41.036111                      318.537500   
4625            44.288000                      309.442500   

      Mean_estimated_fire_radiative_power  Mean_confidence  
0                                0.000000         0.000000  
1                                0.000000         0.000000  
2                                0.000000         0.000000  
3                      

In [48]:
pred_dict['NSW']

,Date,tgt,train,test
0,2005-01-01,0.018144,0.016643,NaN
1,2005-01-02,0.037157,0.037975,NaN
2,2005-01-03,0.033114,0.032679,NaN
3,2005-01-04,0.040383,0.039563,NaN
4,2005-01-05,0.009786,0.009736,NaN
...,...,...,...,...
5646,2020-06-17,0.000588,NaN,0.007489
5647,2020-06-18,0.001459,NaN,0.005660
5648,2020-06-19,0.000000,NaN,0.001714
5649,2020-06-20,0.000642,NaN,0.008879


In [31]:
pred_dict['NSW']

,Date,tgt,train,test
0,2005-01-01,0.001945,0.000966,NaN
1,2005-01-02,0.000000,0.000803,NaN
2,2005-01-03,0.000000,0.000550,NaN
3,2005-01-04,0.002188,0.002586,NaN
4,2005-01-05,0.009635,0.009212,NaN
...,...,...,...,...
5736,2020-09-15,0.000588,NaN,0.006367
5737,2020-09-16,0.001459,NaN,0.005449
5738,2020-09-17,0.000000,NaN,0.002083
5739,2020-09-18,0.000642,NaN,0.003907


In [31]:
ft, target = gen_supervise(wf[wf['Region'] == 'NSW'], lag = 15, forecast = 7)
# ft = pd.DataFrame(ft)

In [66]:
pd.DataFrame(target)

,0,1,2,3,4,5,6
0,7.980000,5.500000,14.305556,8.375,24.20,7.68,4.00000
1,5.500000,14.305556,8.375000,24.200,7.68,4.00,0.00000
2,14.305556,8.375000,24.200000,7.680,4.00,0.00,0.00000
3,8.375000,24.200000,7.680000,4.000,0.00,0.00,4.50000
4,24.200000,7.680000,4.000000,0.000,0.00,4.50,19.81875
...,...,...,...,...,...,...,...
5757,4.945000,0.000000,1.000000,3.600,3.00,0.00,1.21000
5758,0.000000,1.000000,3.600000,3.000,0.00,1.21,3.00000
5759,1.000000,3.600000,3.000000,0.000,1.21,3.00,0.00000
5760,3.600000,3.000000,0.000000,1.210,3.00,0.00,1.32000


In [20]:
target[]

,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Region
0,2005-01-01,0.00000,0.000000,0.000000,0.000000,NSW
1,2005-01-02,0.00000,0.000000,0.000000,0.000000,NSW
2,2005-01-03,0.00000,0.000000,0.000000,0.000000,NSW
3,2005-01-04,8.68000,312.266667,42.400000,78.666667,NSW
4,2005-01-05,16.61125,322.475000,62.362500,85.500000,NSW
...,...,...,...,...,...,...
5778,2020-10-27,1.21000,311.100000,16.800000,78.000000,NSW
5779,2020-10-28,3.00000,323.166667,20.566667,82.333333,NSW
5780,2020-10-29,0.00000,0.000000,0.000000,0.000000,NSW
5781,2020-10-30,1.32000,315.100000,13.700000,76.000000,NSW


In [72]:
def prepared_data(df, split=0.8, lag=30, forecast=30, ft_cols=None):

    if ft_cols == None:
        ft_cols = [ft for ft in df.columns.values if ft not in [
            'Date', 'Region', 'Replaced']]
    ttl_rows = len(df.index.values)
    temp_df = df[ft_cols]

    split_idx = round(split*ttl_rows)
    scaler = MinMaxScaler(feature_range=(0, 1))
    train, test = temp_df.iloc[:split_idx, :], temp_df.iloc[split_idx:, :]

    scaled_train = scaler.fit_transform(train)
    scaled_train_df = pd.DataFrame(scaled_train, columns=ft_cols)
    scaled_test = scaler.transform(test)
    scaled_test_df = pd.DataFrame(scaled_test, columns=ft_cols)

    scaled_data = dict()
    scaled_data['X_train'], scaled_data['y_train'] = gen_supervise(
        scaled_train_df, lag=lag, forecast=forecast, ft_cols=ft_cols)
    scaled_data['X_test'], scaled_data['y_test'] = gen_supervise(
        scaled_test_df, lag=lag, forecast=forecast)

    data = dict()
    data['X_train'], data['y_train'] = gen_supervise(
        train, lag=lag, forecast=forecast, ft_cols=ft_cols)
    data['X_test'], data['y_test'] = gen_supervise(
        test, lag=lag, forecast=forecast, ft_cols=ft_cols)

    return scaler, data, scaled_data, train, test, scaled_train_df, scaled_test_df

In [73]:
scaler, data, scaled_data, train, test, scaled_train_df, scaled_test_df = prepared_data(wf[wf['Region'] == 'NSW'])

In [74]:
scaled_train_df

,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence
0,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000
3,0.004220,0.835356,0.046523,0.786667
4,0.008076,0.862665,0.068426,0.855000
...,...,...,...,...
4621,0.010150,0.843117,0.063937,0.868235
4622,0.014407,0.830764,0.048123,0.841667
4623,0.035169,0.831401,0.084439,0.847941
4624,0.019951,0.852132,0.066352,0.885278


In [75]:
scaled_test_df

,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence
0,0.018045,0.846432,0.054589,0.854828
1,0.004760,0.844694,0.043952,0.878571
2,0.023326,0.839273,0.050599,0.850588
3,0.034621,0.845349,0.222503,0.875769
4,0.045753,0.832440,0.045007,0.888030
...,...,...,...,...
1152,0.000588,0.832235,0.018434,0.780000
1153,0.001459,0.864515,0.022567,0.823333
1154,0.000000,0.000000,0.000000,0.000000
1155,0.000642,0.842936,0.015032,0.760000


In [32]:
data['X_train']

NameError: name 'data' is not defined